```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/smoothing.ipynb>`, or a {download}`python script <converted/smoothing.py>` with code cells.
```

# Tutorial 13: Import / Export (I/O)

Using ``optimap`` you can import, export or convert various video or image file formats acquired with different cameras and software packages. For instance, you can load .rsh videos acquired with SciMedia cameras and save them as Matlab files using the following few lines of code:

In [ ]:
import optimap as om

video = om.load_video('filename_of_input_video.rsh')
video = video[:100] # store only the first 100 frames, optional
om.video.save_video('video.mat', video)

Many more file formats can be imported, saved or exported. More detailed instructions are provided below. To test whether you can load your own example video file (e.g. an .rsh file), make sure that the video is located in the appropriate folder, copy and paste the code above into a python script, modify the filename to match your filename and execute the script.

## Importing Videos

The following file formats can be imported with ``optimap``:

* .tif, .tiff (TIFF)
* .png (PNG)
* .gsd, .gsh (SciMedia MiCAM 05)
* .rsh, .rsm, .rsd (SciMedia MiCAM ULTIMA)
* .dat (MultiRecorder)
* .npy (numpy array)
* .mat (MATLAB), loads the first field in the file

Additional file formats will be added in the future (and upon request). All files can be imported using the same {func}`load_video` function, with which it is also possible to load only a specific number of frames or range of the data (e.g. from a specific frame to another), see below. 

### TIFF Images

``optimap`` can read .tif / .tiff (TIFF) saved as an image stack (a single file) or an image series in a folder (8-bit and 16-bit). Simply provide the filename with the .tif or .tiff ending to load a single TIFF stack:

In [ ]:
video = om.load_video('filename.tif')

or the name of the folder containing the images to load a series of .tif images:

In [ ]:
video = om.load_video('folder_containing_tif_images/')

### PNG Images

``optimap`` can read a series of .png (PNG) images stored in a folder. Simply provide the name of the folder containing the image series.

In [ ]:
video = om.load_video('folder_containing_png_images/')

### SciMedia

``optimap`` can read the different SciMedia file formats, including video files acquired with single- and dual-camera systems. To import Scimedia MiCAM ULTIMA video files you only need to provide the filename with the .rsh ending of the header file (assuming it is located in the same location as the .rsd files):

In [ ]:
video = om.load_video('filename.rsh')

To import Scimedia MiCAM 05 video files you only need to provide the filename with the .gsh ending of the header file:

In [ ]:
video = om.load_video('filename.gsh')

### MultiRecorder

``optimap`` can import .dat video files acquired with MultiRecorder (developed by J. Schröder-Schetelig, Max Planck Institute for Dynamics and Self-Organization).

In [ ]:
video = om.load_video('filename.dat')

### Numpy

Videos stored as numpy arrays can easily be imported using:

In [ ]:
video = om.load_video('filename.npy')

### Matlab

Videos stored as arrays in the Matlab file format (.mat) can easily be imported using:

In [ ]:
video = om.load_video('filename.mat')

It is possible to import only parts of a video, for instance, only 1000 frames and only every 2nd frame starting from frame 100:

In [ ]:
video = om.load_video('filename.npy', start_frame=100, frames=1000, step=2, use_mmap=False)

This option is available for all file formats including image series. Please refer to the Documentation for more information.

## Saving / Exporting Videos

The following file formats can be saved / exported with ``optimap``:

* videos as raw data (numpy or Matlab arrays)
* rendered videos (e.g. .mp4 videos)
* images as raw data (numpy or Matlab arrays)
* multiple videos rendered next to each other (e.g. .mp4 videos)
* image sequences in a folder

``optimap`` distinguishes saving and exporting files. Saving files saves them as raw data (e.g. three-dimensional arrays), while exporting renders videos for visualization purposes (such as a .mp4 file). Saving videos prevents data loss (e.g. floating point or integer precision is preserved) and one can reload and continue to process the saved videos later. Exporting data is associated with data loss (e.g. loss of floating point precision) and compression because the video is rendered using a video encoder.

### Saving Videos

The default mode in ``optimap`` is to save videos as arrays in numpy (.npy) or Matlab (.mat) file formats. This saves the data in its rawest form without information loss:

In [ ]:
om.save_video('video.npy',video)

In [ ]:
om.save_video('video.mat',video)

The videos can later be imported and further processed or used in other applications. The {func}`save_video` function preserves the data type (e.g. floating point precision).

Another way to save video data is to save them as a TIFF or PNG image series in a folder (here called ``"my_folder"``):

In [ ]:
om.save_image_sequence(video, filepattern="frame-{:04d}", directory="my_folder", suffix=".tif")

In [ ]:
om.save_image_sequence(video, filepattern="frame-{:04d}", directory="my_folder", suffix=".png")

The resulting images will have the same dimension as the video (e.g. 100x100 or 128x128 pixels). The `filepattern` can be modified, e.g. the `"frame-"` can be left out to obtain only zero-padded numbers (you want to change `:04d` to `:05d` if your video has more than 9999 frames). If the videos are 8- or 16-bit, then the resulting .tif or .png images will automatically be 8- or 16-bit, correspondingly. Accordingly, it is possible to prevent data loss if your raw video is 16-bit. If the images are all black, all white, show salt and pepper noise or look weird, then you need make sure that your video contains values which are normalized and formatted correctly. For instance, 8-bit videos can only contain integer values between 0-255. You can use the code snippet below to format your video data before saving it as an image series:

In [ ]:
import numpy as np

om.print_properties(video)
video = video.astype(np.float32)
video = (video-np.min(video))/np.max(video)*255 # alternatively *65536 for 16-bit
om.print_properties(video)
video = video.astype(np.uint8) # alternatively np.uint16 for 16-bit
om.print_properties(video)

With the normalization (the subtraction of the minimum and the division by the maximum) in the code snippet above it is ensured that no values are below 0 or above 255 (or 65536, respectively).

### Video Export

The main purpose of exporting videos is to generate or render videos in a file format (.mp4) that can be played with an external video player application (e.g. Quicktime, VLC, Windows Media Player etc.). You can export videos in several ways: 

* a single grayscale video (e.g. showing the original or normalized video)
* a processed video (e.g. phase maps) with a special colormap (e.g. hsv, jet, magma, etc.)
* an overlay of 2 videos on top of each other (e.g. original grayscale video plus calcium waves highlighted in a particular color)
* multiple videos rendered next to each other (e.g. original grayscale video plus motion-stabilized video)

The most straight-forward way to export a video is:

In [ ]:
om.export_video('video.mp4', video)

This will generate a rendered .mp4 video containing the entire video data at a framerate of 60fps and at the original resolution (e.g. 100x100 or 128x128 pixels). You can customize the video export as follows:

In [ ]:
om.export_video("video.mp4", video[123:323], fps=30, skip_frames=2, vmin=0.1, vmax=0.9, cmap="viridis")

Here, only every 2nd frame from frames 123-323 are exported at a framerate of 30fps, `vmin` and `vmax` define the dynamic range of pixel values (0.1 is black and 0.9 white with the grayscale colormap) and `viridis` is a different colormap than just a grayscale colormap.

You can overlay a processed video, for instance, a pixel-wise normalized video which shows action potential or calcium waves, see [Tutorial 2](signal_extraction.ipynb), on top of the original grayscale video as follows:

In [ ]:
video_waves = om.video.normalize_pixelwise(video)
om.export_video_with_overlay("video.mp4", video, video_waves)

```{warning}
This tutorial is currently work in progress. We will add more information soon.
```